In [1]:
#necessary imports
from binance.client import Client
import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [2]:
#client for work with binance
client = Client()

#all history for model 
lookback_days = 60

#8 hours previous information for each prediction
lookback_interval = 16

#bitcoin
symbol = 'BTCUSDT'

#information with a price in 30 minute steps
interval = client.KLINE_INTERVAL_30MINUTE    

#initialize dataframe with data from binance
all_data = pd.DataFrame()

#start and end time for 60 days history
end_time = pd.Timestamp.now()
start_time = end_time - pd.Timedelta(days = lookback_days)

while start_time < end_time:
    start_str = str(int(start_time.timestamp() * 1000))

    #take 1000 lines of information
    klines = client.get_historical_klines(symbol, interval, start_str, limit = 1000)

    if not klines:
        break

    #create dataframe with data
    data = pd.DataFrame(klines, columns = [
                                            'timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 
                                            'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume',
                                            'taker_buy_quote_asset_volume', 'ignore'
                                          ])

    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    data.set_index('timestamp', inplace=True)
    data = data.apply(pd.to_numeric, errors = 'coerce')

    all_data = pd.concat([all_data, data])

    #last datetime
    start_time = data.index[-1] + pd.Timedelta(minutes = 5)

    time.sleep(0.5)
    
#sort data by timestamp
btc = all_data.sort_index().ffill()

In [3]:
#example of bitcoin data with prices
btc

,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
timestamp,,,,,,,,,,,
2025-02-15 22:30:00,97450.01,97566.06,97432.50,97545.76,137.08232,1739660399999,1.336402e+07,47082,69.97286,6.821113e+06,0
2025-02-15 23:00:00,97545.77,97624.00,97426.56,97591.36,250.77348,1739662199999,2.444940e+07,63457,126.42784,1.232573e+07,0
2025-02-15 23:30:00,97591.35,97634.22,97519.24,97569.66,102.87179,1739663999999,1.003692e+07,26596,37.43649,3.652243e+06,0
2025-02-16 00:00:00,97569.67,97700.88,97554.65,97674.97,74.99016,1739665799999,7.321794e+06,24522,41.51239,4.052672e+06,0
2025-02-16 00:30:00,97674.97,97704.47,97584.90,97594.57,71.66860,1739667599999,6.998170e+06,19730,28.50778,2.783864e+06,0
...,...,...,...,...,...,...,...,...,...,...,...
2025-04-16 17:00:00,85048.54,85481.16,85022.01,85304.37,536.19474,1744824599999,4.571748e+07,83115,323.85310,2.761330e+07,0
2025-04-16 17:30:00,85304.37,85500.00,83275.00,83281.21,2128.39829,1744826399999,1.792253e+08,252444,837.64934,7.055313e+07,0
2025-04-16 18:00:00,83281.21,84472.31,83276.00,84386.81,1437.97262,1744828199999,1.205844e+08,247700,866.98346,7.266248e+07,0


In [4]:
#split data to train and test
test_percent = 0.2
train_size = int(len(btc) * (1 - test_percent))

train_data = btc.iloc[:train_size]
test_data = btc.iloc[train_size:]

In [5]:
#scaling data
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(train_data[['close']])
scaled_test = scaler.transform(test_data[['close']])

In [6]:
#create lists of data to prediction and lists with correct predictions to train LSTM model
x_train = []
y_train = []
x_test = []
y_test = []

for i in range(lookback_interval, len(scaled_train)):
    x_train.append(scaled_train[i-lookback_interval:i])
    y_train.append(scaled_train[i])

for i in range(lookback_interval, len(scaled_test)):
    x_test.append(scaled_test[i-lookback_interval:i])
    y_test.append(scaled_test[i])   

In [7]:
#convert lists to np arrays
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [8]:
#create LSTM model
model = Sequential()

model.add(LSTM(96, input_shape=(lookback_interval, 1), return_sequences=True, kernel_initializer = 'glorot_uniform'))
model.add(Dropout(0.2))

model.add(LSTM(64))
model.add(Dropout(0.3))

model.add(Dense(1, activation = 'relu'))

#compile LSTM model
model.compile(optimizer='adam', loss='mse')

#train LSTM model
model.fit(x_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

#calculate mean error on test data
mse = model.evaluate(x_test, y_test)
print(f"Mean Squared Error: {mse}")

Epoch 1/20


D:\Users\userr\anaconda3\envs\py311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.0477 - val_loss: 0.0012
Epoch 2/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0044 - val_loss: 0.0014
Epoch 3/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0038 - val_loss: 0.0010
Epoch 4/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0041 - val_loss: 0.0011
Epoch 5/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0036 - val_loss: 0.0011
Epoch 6/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0037 - val_loss: 0.0020
Epoch 7/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0043 - val_loss: 8.7063e-04
Epoch 8/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0030 - val_loss: 8.7763e-04
Epoch 9/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0028 - val_loss: 0.0010
Epoch 10/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0030 - val_loss: 8.1183e-04
Epoch 11/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0027 - val_loss: 0.0012
Epoch 12/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.

In [9]:
#compare predictions with real data
predictions = model.predict(x_test)
predictions_inv = scaler.inverse_transform(predictions)
print(predictions_inv[0:5])
print(predictions_inv[554:559])
print(test_data[['close']][lookback_interval:])

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step 
[[83958.19 ]
 [83926.81 ]
 [83888.58 ]
 [83879.28 ]
 [83862.305]]
[[84911.56]
 [85060.  ]
 [84874.  ]
 [84711.15]
 [84530.76]]
                        close
timestamp                    
2025-04-05 04:00:00  83838.19
2025-04-05 04:30:00  83677.75
2025-04-05 05:00:00  83850.00
2025-04-05 05:30:00  83688.67
2025-04-05 06:00:00  83617.99
...                       ...
2025-04-16 17:00:00  85304.37
2025-04-16 17:30:00  83281.21
2025-04-16 18:00:00  84386.81
2025-04-16 18:30:00  84029.97
2025-04-16 19:00:00  84165.99

[559 rows x 1 columns]


In [10]:
#calculate mean prediction error on all historical data
historical_data = btc
scaled_historical = scaler.transform(historical_data[['close']])
x_historical = []
y_historical = []

for i in range(lookback_interval, len(scaled_historical)):
    x_historical.append(scaled_historical[i-lookback_interval:i])
    y_historical.append(scaled_historical[i])
x_historical = np.array(x_historical)
y_historical = np.array(y_historical)
historical_error = []
for i in range(0, len(y_historical)):
    scaled_prediction = model.predict(np.array([x_historical[i]]),verbose=None)
    prediction = scaler.inverse_transform(scaled_prediction)[0][0]
    error = abs(prediction - (scaler.inverse_transform([y_historical[i]])[0][0]))
    historical_error.append(error)
np.array(historical_error).mean()

np.float64(393.5349126355843)

In [12]:
#Reading data for last 8 hour for actual price prediction
client = Client()


lookback_interval = 16
symbol = 'BTCUSDT'
interval = client.KLINE_INTERVAL_30MINUTE   

all_data = pd.DataFrame()
end_time = pd.Timestamp.utcnow().tz_localize(None) 
start_time = end_time - pd.Timedelta(hours = 8)
while start_time < end_time:
    start_str = str(int(start_time.timestamp() * 1000))

    klines = client.get_historical_klines(symbol, interval, start_str, limit = 1000)

    if not klines:
        break

    data = pd.DataFrame(klines, columns = [
                                            'timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 
                                            'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume',
                                            'taker_buy_quote_asset_volume', 'ignore'
                                          ])

    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    data.set_index('timestamp', inplace=True)
    data = data.apply(pd.to_numeric, errors = 'coerce')

    all_data = pd.concat([all_data, data])

    start_time = data.index[-1] + pd.Timedelta(minutes = 5)

    time.sleep(0.5)

btc = all_data.sort_index().ffill()

In [13]:
#last 8 hours data example
btc

,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
timestamp,,,,,,,,,,,
2025-04-16 11:30:00,83873.83,84127.05,83782.60,84048.11,203.06508,1744804799999,1.705654e+07,32579,110.09497,9.247317e+06,0
2025-04-16 12:00:00,84048.36,84262.00,84000.00,84180.65,306.75831,1744806599999,2.580588e+07,45521,178.11152,1.498462e+07,0
2025-04-16 12:30:00,84180.66,84213.53,83722.56,83757.72,353.91624,1744808399999,2.970409e+07,50130,137.08467,1.151072e+07,0
2025-04-16 13:00:00,83757.73,83800.28,83620.75,83725.66,256.32885,1744810199999,2.145699e+07,37715,111.84216,9.361987e+06,0
2025-04-16 13:30:00,83725.67,84204.54,83512.20,83736.37,836.36482,1744811999999,7.017404e+07,134340,466.18532,3.912162e+07,0
2025-04-16 14:00:00,83736.36,84460.00,83714.31,84357.99,586.73049,1744813799999,4.939004e+07,95620,331.22164,2.787812e+07,0
2025-04-16 14:30:00,84357.99,84849.06,84263.98,84605.31,728.08601,1744815599999,6.157678e+07,101357,353.04618,2.986416e+07,0
2025-04-16 15:00:00,84605.30,85270.27,84502.02,85105.85,905.42217,1744817399999,7.692503e+07,109935,517.46774,4.396367e+07,0
2025-04-16 15:30:00,85105.85,85171.59,84785.51,84846.58,613.42243,1744819199999,5.211529e+07,94193,216.13854,1.836280e+07,0


In [14]:
#predict actual price
scaled_btc = scaler.transform(btc[['close']])
scaled_btc = np.array(scaled_btc)
scaled_prediction = model.predict(np.array([scaled_btc]))
prediction = scaler.inverse_transform(scaled_prediction)[0][0]
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


np.float32(84362.96)

In [15]:
#compare with real price
btc.iloc[:]['close'][-1]

C:\Users\userr\AppData\Local\Temp\ipykernel_11812\2370184923.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  btc.iloc[:]['close'][-1]


np.float64(83972.01)

In [ ]:
#simple trading strategy: 
#we have 1000$ at initial state
#we spend all money to buy bitcoin when model predict that price will increase more than to fees (one for buy and another one for sell) of actual price
#than we sell all bitcoins when model predict than price will decrease
#we collect data and predict new price every thirty minutes
money = 1000
coins = 0

want_buy_flg = 1

money_list = [1000]
coins_list = [0]

real_price = []
predicted_price = [None]

fee_percent = 0.001

while True:
    client = Client()

    lookback_interval = 16
    symbol = 'BTCUSDT'
    interval = client.KLINE_INTERVAL_30MINUTE   
    
    all_data = pd.DataFrame()
    end_time = pd.Timestamp.utcnow().tz_localize(None) 
    start_time = end_time - pd.Timedelta(hours = 8)
    while start_time < end_time:
        start_str = str(int(start_time.timestamp() * 1000))
    
        klines = client.get_historical_klines(symbol, interval, start_str, limit = 1000)
    
        if not klines:
            break
    
        data = pd.DataFrame(klines, columns = [
                                                'timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 
                                                'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume',
                                                'taker_buy_quote_asset_volume', 'ignore'
                                              ])
    
        data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
        data.set_index('timestamp', inplace=True)
        data = data.apply(pd.to_numeric, errors = 'coerce')
    
        all_data = pd.concat([all_data, data])
    
        start_time = data.index[-1] + pd.Timedelta(minutes = 5)
    
        time.sleep(0.5)
    
    btc = all_data.sort_index().ffill()
    
    real_price.append(float(btc.iloc[:]['close'][-1]))

    print(f'Real price list: {real_price}')
    print(f'Predicted price list: {predicted_price}')
    
    scaled_btc = scaler.transform(btc[['close']])
    scaled_btc = np.array(scaled_btc)
    scaled_prediction = model.predict(np.array([scaled_btc]), verbose = None)
    prediction = scaler.inverse_transform(scaled_prediction)[0][0]

    predicted_price.append(float(prediction))


    print(f'Prediction: {float(prediction)}')

    if (predicted_price[-1] * (1.00 - (2 * fee_percent))  > real_price[-1]) and want_buy_flg == 1:
        coins = (money / real_price[-1]) * (1.00 - fee_percent)
        coins_list.append(coins)
        money = 0
        money_list.append(money)
        want_buy_flg = 0

    if (predicted_price[-1]  < real_price[-1]) and want_buy_flg == 0:
        money = (real_price[-1] * coins) * (1.00 - fee_percent)
        money_list.append(money)
        coins = 0
        coins_list.append(coins)
        want_buy_flg = 1

    print(f'Money list: {money_list}')
    print(f'Coins list: {coins_list}')

    print('\n')
    print('-----------------------')
    print('\n')

    for i in range(150):
        time.sleep(12)

C:\Users\userr\AppData\Local\Temp\ipykernel_11812\1969906219.py:50: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  real_price.append(float(btc.iloc[:]['close'][-1]))


Real price list: [83900.0]
Predicted price list: [None]
Prediction: 84351.8671875
Money list: [1000, 0]
Coins list: [0, 0.011907032181168056]


-----------------------




C:\Users\userr\AppData\Local\Temp\ipykernel_11812\1969906219.py:50: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  real_price.append(float(btc.iloc[:]['close'][-1]))


Real price list: [83900.0, 84090.9]
Predicted price list: [None, 84351.8671875]
Prediction: 84220.375
Money list: [1000, 0]
Coins list: [0, 0.011907032181168056]


-----------------------




C:\Users\userr\AppData\Local\Temp\ipykernel_11812\1969906219.py:50: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  real_price.append(float(btc.iloc[:]['close'][-1]))


...

Real price list: [83900.0, 84090.9, 84399.99, 84474.0, 84381.71, 84408.79, 84430.0, 84489.61, 84355.99, 84160.0, 84002.01, 84171.48, 84248.49, 84299.64, 84200.86, 84121.77, 84165.93, 83932.01, 83922.43, 84024.81, 84242.36, 84459.99, 84443.33, 84815.0, 84395.82, 84486.58, 84325.02, 84355.12, 84428.66, 84576.01, 84814.0, 84776.0, 84614.0, 84424.0]
Predicted price list: [None, 84351.8671875, 84220.375, 84252.1015625, 84343.3984375, 84362.515625, 84454.2421875, 84528.1484375, 84551.390625, 84546.4609375, 84476.671875, 84361.6953125, 84297.5859375, 84278.7890625, 84311.5390625, 84327.0546875, 84314.9609375, 84334.4765625, 84213.6484375, 84161.109375, 84134.0625, 84175.9375, 84256.0859375, 84326.5546875, 84495.4765625, 84713.5390625, 84656.3515625, 84588.4921875, 84537.1328125, 84511.171875, 84525.328125, 84589.375, 84671.578125, 84720.953125]
Prediction: 84721.7734375
Money list: [1000, 0, 1003.9484436232419, 0, 1000.5919059419107, 0, 1000.7237743154508, 0]
Coins list: [0, 0.011907032181168

C:\Users\userr\AppData\Local\Temp\ipykernel_11812\1969906219.py:50: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  real_price.append(float(btc.iloc[:]['close'][-1]))


Real price list: [83900.0, 84090.9, 84399.99, 84474.0, 84381.71, 84408.79, 84430.0, 84489.61, 84355.99, 84160.0, 84002.01, 84171.48, 84248.49, 84299.64, 84200.86, 84121.77, 84165.93, 83932.01, 83922.43, 84024.81, 84242.36, 84459.99, 84443.33, 84815.0, 84395.82, 84486.58, 84325.02, 84355.12, 84428.66, 84576.01, 84814.0, 84776.0, 84614.0, 84424.0, 84031.38]
Predicted price list: [None, 84351.8671875, 84220.375, 84252.1015625, 84343.3984375, 84362.515625, 84454.2421875, 84528.1484375, 84551.390625, 84546.4609375, 84476.671875, 84361.6953125, 84297.5859375, 84278.7890625, 84311.5390625, 84327.0546875, 84314.9609375, 84334.4765625, 84213.6484375, 84161.109375, 84134.0625, 84175.9375, 84256.0859375, 84326.5546875, 84495.4765625, 84713.5390625, 84656.3515625, 84588.4921875, 84537.1328125, 84511.171875, 84525.328125, 84589.375, 84671.578125, 84720.953125, 84721.7734375]
Prediction: 84637.5625
Money list: [1000, 0, 1003.9484436232419, 0, 1000.5919059419107, 0, 1000.7237743154508, 0]
Coins list:

C:\Users\userr\AppData\Local\Temp\ipykernel_11812\1969906219.py:50: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  real_price.append(float(btc.iloc[:]['close'][-1]))


Real price list: [83900.0, 84090.9, 84399.99, 84474.0, 84381.71, 84408.79, 84430.0, 84489.61, 84355.99, 84160.0, 84002.01, 84171.48, 84248.49, 84299.64, 84200.86, 84121.77, 84165.93, 83932.01, 83922.43, 84024.81, 84242.36, 84459.99, 84443.33, 84815.0, 84395.82, 84486.58, 84325.02, 84355.12, 84428.66, 84576.01, 84814.0, 84776.0, 84614.0, 84424.0, 84031.38, 84073.99]
Predicted price list: [None, 84351.8671875, 84220.375, 84252.1015625, 84343.3984375, 84362.515625, 84454.2421875, 84528.1484375, 84551.390625, 84546.4609375, 84476.671875, 84361.6953125, 84297.5859375, 84278.7890625, 84311.5390625, 84327.0546875, 84314.9609375, 84334.4765625, 84213.6484375, 84161.109375, 84134.0625, 84175.9375, 84256.0859375, 84326.5546875, 84495.4765625, 84713.5390625, 84656.3515625, 84588.4921875, 84537.1328125, 84511.171875, 84525.328125, 84589.375, 84671.578125, 84720.953125, 84721.7734375, 84637.5625]
Prediction: 84516.734375
Money list: [1000, 0, 1003.9484436232419, 0, 1000.5919059419107, 0, 1000.72377